### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [15]:
import os
from langchain.chat_models import init_chat_model

# os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]="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"
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re birds, right? Some species are known for mimicking human speech, like the African Grey Parrot. But why do they do that? Maybe it\'s a form of communication? But communication for what purpose?\n\nFirst, I should think about their natural behavior. In the wild, parrots have complex social structures. They live in flocks, so maybe they use vocalizations to interact with each other. But how does that relate to mimicking humans? Perhaps in captivity, they mimic humans because they associate certain sounds with positive outcomes, like getting food or attention. But is there a deeper reason?\n\nI remember reading that some parrots, like the budgerigar (budgie), can learn to mimic words, but maybe the African Grey is more advanced. Is there a cognitive aspect here? Maybe they have a high level of intelligence, similar to some primates. If they\'re intellige

In [16]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"

# This is not an agent, It can't execute tools on its own.
model_with_tools=model.bind_tools([get_weather])

In [17]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. So I'll call the function with location set to Boston. Make sure the JSON is correctly formatted. Let me structure the tool_call with the function name and the arguments.\n", 'tool_calls': [{'id': 'sc6rjnn39', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 154, 'total_tokens': 252, 'completion_time': 0.153163246, 'completion_tokens_details': {'reasoning_tokens': 74}, 'prompt_time': 0.006094139, 'prompt_tokens_details': None, 'queue_time': 0.15809941, 'total_time': 0.159257385}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'mo

### Tool Execution Loops

In [ ]:
# manual implementation of an agent loop.
# You are doing by hand what create_agent() normally does for you automatically.

# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny right now. It's a great day to enjoy outdoor activities! 🌞


In [19]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified Boston, I need to call this function with "Boston" as the location. I\'ll make sure to format the tool call correctly within the XML tags as instructed.\n', 'tool_calls': [{'id': '0a6rj4ar4', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 153, 'total_tokens': 246, 'completion_time': 0.134260093, 'completion_tokens_details': {'reasoning_tokens': 69}, 'prompt_time': 0.015243927, 'prompt_tokens_details': None, 'queue_time': 0.057360903, 'total_time': 0.14950402}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand